# TimesFM 実行確認ノート（網羅・再帰・段階）

このノートは `outputs/html/cle_v6_full_timesfm_20260211_182948.html` を基準に、
`Modules / Classes / Functions / External / Methods/Props` を章立てして、
項目単位で実行確認します。


## 0. 実装計画

1. 解析レポートを読み込み、カテゴリ別テーブルを固定化する。  
2. 再帰参照（Parent/Path）で項目単位の識別子を正規化する。  
3. 章ごとに `md + code` を分離し、カテゴリごとに実行関数を定義する。  
4. 各カテゴリで「全件実行セル」と「単体深掘りセル」を用意する。  
5. 結果を `docs/timesfm_*_check_results.csv` として保存する。  
6. 失敗時は例外を握って `status/error_type/error_message` を残す。


## 1. 初期化とレポート読込


In [1]:
from pathlib import Path
import importlib
import inspect
import os
import sys
from typing import Any, Dict

import pandas as pd
import numpy as np

CWD = Path.cwd().resolve()
ROOT = CWD
for p in [CWD, *CWD.parents]:
    if (p / 'src').exists() and (p / 'outputs').exists():
        ROOT = p
        break
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

HTML_REPORT = ROOT / 'outputs/html/cle_v6_full_timesfm_20260211_182948.html'
DOCS_DIR = ROOT / 'src/model/timesfm/docs'
DOCS_DIR.mkdir(parents=True, exist_ok=True)

print('ROOT =', ROOT)
print('HTML_REPORT =', HTML_REPORT)
print('HTML exists =', HTML_REPORT.exists())

from IPython.display import display


ROOT = /mnt/e/env/ts/lib_ana
HTML_REPORT = /mnt/e/env/ts/lib_ana/outputs/html/cle_v6_full_timesfm_20260211_182948.html
HTML exists = True


In [2]:
tables = pd.read_html(HTML_REPORT)
summary_df = tables[0]
modules_df = tables[1].copy()
classes_df = tables[2].copy()
functions_df = tables[3].copy()
methods_df = tables[4].copy()
externals_df = tables[5].copy()
nodes_df = tables[8].copy()
edges_df = tables[9].copy()

print('summary/modules/classes/functions/methods/externals/nodes/edges =')
print(len(summary_df), len(modules_df), len(classes_df), len(functions_df), len(methods_df), len(externals_df), len(nodes_df), len(edges_df))
summary_df


summary/modules/classes/functions/methods/externals/nodes/edges =
11 9 21 22 76 58 186 241


,Metric,Value
0,library,timesfm
1,root_modules,[timesfm]
2,files_parsed,9
3,modules,9
4,classes,21
5,functions,22
6,methods,76
7,external,58
8,nodes,186
9,edges,241


In [3]:
coverage = {
    'modules': int((nodes_df['Type'] == 'module').sum()),
    'classes': int((nodes_df['Type'] == 'class').sum()),
    'functions': int((nodes_df['Type'] == 'function').sum()),
    'methods': int((nodes_df['Type'] == 'method').sum()),
    'externals': int((nodes_df['Type'] == 'external').sum()),
    'nodes_total': int(len(nodes_df)),
    'edges_total': int(len(edges_df)),
}
coverage


{'modules': 9,
 'classes': 21,
 'functions': 22,
 'methods': 76,
 'externals': 58,
 'nodes_total': 186,
 'edges_total': 241}

## 2. 共通ユーティリティ（再帰解決・安全実行）


In [4]:
node_lookup = nodes_df[['ID', 'Type', 'Name', 'Path', 'Module', 'Parent']].copy()
node_map = {row.ID: row for row in node_lookup.itertuples(index=False)}

OPTIONAL_MODULE_PREFIX_DEPS = {
    'timesfm.timesfm_jax': {'paxml'},
    'timesfm.patched_decoder': {'praxis', 'lingvo'},
}
OPTIONAL_IMPORT_TARGETS = {'paxml', 'praxis', 'praxis.layers', 'lingvo'}

def resolve_chain(node_id: str):
    chain = []
    cur = node_id
    seen = set()
    while isinstance(cur, str) and cur and cur in node_map and cur not in seen:
        seen.add(cur)
        row = node_map[cur]
        chain.append((row.Type, row.Name, row.Path))
        cur = row.Parent
    return list(reversed(chain))

def _optional_dep_for(module_path: str, missing_name: str | None):
    if module_path in OPTIONAL_IMPORT_TARGETS:
        return missing_name or module_path
    for prefix, deps in OPTIONAL_MODULE_PREFIX_DEPS.items():
        if module_path.startswith(prefix):
            if missing_name is None or missing_name in deps:
                return missing_name or ','.join(sorted(deps))
    return None

def safe_import(module_path: str):
    import io
    import contextlib
    import warnings

    buf_out = io.StringIO()
    buf_err = io.StringIO()
    try:
        with warnings.catch_warnings(record=True) as ws:
            warnings.simplefilter('always')
            with contextlib.redirect_stdout(buf_out), contextlib.redirect_stderr(buf_err):
                mod = importlib.import_module(module_path)
        return {
            'status': 'ok',
            'module': mod,
            'error_type': None,
            'error_message': None,
            'missing_optional_dep': None,
            'captured_stdout': buf_out.getvalue()[:2000],
            'captured_stderr': buf_err.getvalue()[:2000],
            'captured_warnings': [str(w.message)[:300] for w in ws],
        }
    except Exception as exc:  # noqa: BLE001
        missing_name = getattr(exc, 'name', None) if isinstance(exc, ModuleNotFoundError) else None
        opt = _optional_dep_for(module_path, missing_name)
        status = 'skipped_optional_dependency' if opt else 'error'
        return {
            'status': status,
            'module': None,
            'error_type': type(exc).__name__,
            'error_message': str(exc),
            'missing_optional_dep': opt,
            'captured_stdout': buf_out.getvalue()[:2000],
            'captured_stderr': buf_err.getvalue()[:2000],
            'captured_warnings': [],
        }

def parse_class_and_member(path: str):
    parts = path.split('.')
    if len(parts) < 3:
        return None, None, None
    module = '.'.join(parts[:-2])
    cls = parts[-2]
    member = parts[-1]
    return module, cls, member


## 3. Modules 章


### 3.1 Modules 全件実行

各 module について import 可否、`__file__`、docstring 先頭、配下 entity 件数を検証します。


In [5]:
def run_module_checks(df: pd.DataFrame) -> pd.DataFrame:
    rows = []
    for r in df.itertuples(index=False):
        module_path = str(r.Module)
        imp = safe_import(module_path)
        rec = {
            'id': r.ID,
            'name': r.Name,
            'module': module_path,
            'path': r.Path,
            'status': imp['status'],
            'error_type': imp['error_type'],
            'error_message': imp['error_message'],
            'missing_optional_dep': imp.get('missing_optional_dep'),
            'file': None,
            'doc_head': None,
            'entities_in_report': int((nodes_df['Module'] == module_path).sum()),
        }
        if imp['module'] is not None:
            mod = imp['module']
            rec['file'] = getattr(mod, '__file__', None)
            rec['doc_head'] = ((getattr(mod, '__doc__', '') or '').strip().split("\n")[0])[:180]
        rows.append(rec)
    return pd.DataFrame(rows)

module_results = run_module_checks(modules_df)
module_results.to_csv(DOCS_DIR / 'timesfm_module_check_results.csv', index=False)
module_results


SyntaxError: unterminated string literal (detected at line 21) (410078088.py, line 21)

### 3.2 Modules 単体深掘り


In [6]:
module_target = 'timesfm.timesfm_base'
module_results[module_results['module'] == module_target]


NameError: name 'module_results' is not defined

## 4. Classes 章


### 4.1 Classes 全件実行

各 class について import 解決、継承、公開メンバー数、property数を検証します。


In [7]:
def resolve_class(path: str):
    parts = path.split('.')
    module_path = '.'.join(parts[:-1])
    class_name = parts[-1]
    imp = safe_import(module_path)
    if imp['module'] is None:
        return imp, None
    try:
        cls = getattr(imp['module'], class_name)
        return imp, cls
    except Exception as exc:  # noqa: BLE001
        return {
            'status': 'error',
            'module': imp['module'],
            'error_type': type(exc).__name__,
            'error_message': str(exc),
        }, None


def run_class_checks(df: pd.DataFrame) -> pd.DataFrame:
    rows = []
    for r in df.itertuples(index=False):
        imp, cls = resolve_class(str(r.Path))
        rec = {
            'id': r.ID,
            'path': r.Path,
            'module': r.Module,
            'class_name': r.Name,
            'status': imp['status'],
            'error_type': imp.get('error_type'),
            'error_message': imp.get('error_message'),
            'mro': None,
            'public_members': None,
            'property_count': None,
        }
        if cls is not None:
            members = [n for n in dir(cls) if not n.startswith('_')]
            prop_count = 0
            for n, v in inspect.getmembers(cls):
                if isinstance(v, property):
                    prop_count += 1
            rec['mro'] = ' -> '.join([c.__name__ for c in cls.__mro__[:5]])
            rec['public_members'] = len(members)
            rec['property_count'] = prop_count
        rows.append(rec)
    return pd.DataFrame(rows)

class_results = run_class_checks(classes_df)
class_results.to_csv(DOCS_DIR / 'timesfm_class_check_results.csv', index=False)
class_results.head(30)


 See https://github.com/google-research/timesfm/blob/master/README.md for updated APIs.
Loaded PyTorch TimesFM, likely because python version is 3.11.14 (main, Oct 21 2025, 18:31:21) [GCC 11.2.0].


/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/scipy/__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 2.4.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.4.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/az

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.4.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 758, in start
    self.io_loop.start()
  Fi

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.4.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 758, in start
    self.io_loop.start()
  Fi

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.4.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 758, in start
    self.io_loop.start()
  Fi

AttributeError: _ARRAY_API not found

,id,path,module,class_name,status,error_type,error_message,mro,public_members,property_count
0,2,timesfm.data_loader.TimeSeriesdata,timesfm.data_loader,TimeSeriesdata,error,ImportError,numpy.core.multiarray failed to import,None,NaN,NaN
1,17,timesfm.time_features.TimeCovariates,timesfm.time_features,TimeCovariates,error,ImportError,numpy.core.multiarray failed to import,None,NaN,NaN
2,38,timesfm.pytorch_patched_decoder.TimesFMConfig,timesfm.pytorch_patched_decoder,TimesFMConfig,ok,None,None,TimesFMConfig -> object,13.0,0.0
3,46,timesfm.pytorch_patched_decoder.ResidualBlock,timesfm.pytorch_patched_decoder,ResidualBlock,ok,None,None,ResidualBlock -> Module -> object,49.0,0.0
4,49,timesfm.pytorch_patched_decoder.RMSNorm,timesfm.pytorch_patched_decoder,RMSNorm,ok,None,None,RMSNorm -> Module -> object,49.0,0.0
5,53,timesfm.pytorch_patched_decoder.TransformerMLP,timesfm.pytorch_patched_decoder,TransformerMLP,ok,None,None,TransformerMLP -> Module -> object,49.0,0.0
6,56,timesfm.pytorch_patched_decoder.TimesFMAttention,timesfm.pytorch_patched_decoder,TimesFMAttention,ok,None,None,TimesFMAttention -> Module -> object,49.0,0.0
7,60,timesfm.pytorch_patched_decoder.TimesFMDecoder...,timesfm.pytorch_patched_decoder,TimesFMDecoderLayer,ok,None,None,TimesFMDecoderLayer -> Module -> object,49.0,0.0
8,63,timesfm.pytorch_patched_decoder.StackedDecoder,timesfm.pytorch_patched_decoder,StackedDecoder,ok,None,None,StackedDecoder -> Module -> object,49.0,0.0
9,66,timesfm.pytorch_patched_decoder.PositionalEmbe...,timesfm.pytorch_patched_decoder,PositionalEmbedding,ok,None,None,PositionalEmbedding -> Module -> object,49.0,0.0


### 4.2 Classes 単体深掘り


In [8]:
class_target = 'timesfm.timesfm_base.TimesFmBase'
class_results[class_results['path'] == class_target]


,id,path,module,class_name,status,error_type,error_message,mro,public_members,property_count
20,169,timesfm.timesfm_base.TimesFmBase,timesfm.timesfm_base,TimesFmBase,ok,None,None,TimesFmBase -> object,4.0,0.0


## 5. Functions 章


### 5.1 Functions 全件実行

各 function について import 解決、callable判定、シグネチャ、無引数実行可否を検証します。


In [9]:
def resolve_function(path: str):
    parts = path.split('.')
    module_path = '.'.join(parts[:-1])
    fn_name = parts[-1]
    imp = safe_import(module_path)
    if imp['module'] is None:
        return imp, None
    try:
        fn = getattr(imp['module'], fn_name)
        return imp, fn
    except Exception as exc:  # noqa: BLE001
        return {
            'status': 'error',
            'module': imp['module'],
            'error_type': type(exc).__name__,
            'error_message': str(exc),
        }, None


def callable_without_required_args(obj) -> bool:
    try:
        sig = inspect.signature(obj)
    except Exception:
        return False
    for p in sig.parameters.values():
        if p.kind in (p.POSITIONAL_ONLY, p.POSITIONAL_OR_KEYWORD, p.KEYWORD_ONLY):
            if p.default is inspect._empty:
                return False
    return True


def run_function_checks(df: pd.DataFrame) -> pd.DataFrame:
    rows = []
    for r in df.itertuples(index=False):
        imp, fn = resolve_function(str(r.Path))
        rec = {
            'id': r.ID,
            'path': r.Path,
            'module': r.Module,
            'function_name': r.Name,
            'status': imp['status'],
            'error_type': imp.get('error_type'),
            'error_message': imp.get('error_message'),
            'is_callable': None,
            'signature': None,
            'no_required_args': None,
            'invoke_status': 'not_tried',
            'invoke_error': None,
        }
        if fn is not None:
            rec['is_callable'] = callable(fn)
            try:
                rec['signature'] = str(inspect.signature(fn))
            except Exception:
                rec['signature'] = None
            no_req = callable_without_required_args(fn)
            rec['no_required_args'] = bool(no_req)
            if no_req:
                try:
                    _ = fn()
                    rec['invoke_status'] = 'ok'
                except Exception as exc:  # noqa: BLE001
                    rec['invoke_status'] = 'error'
                    rec['invoke_error'] = f'{type(exc).__name__}: {exc}'
        rows.append(rec)
    return pd.DataFrame(rows)

function_results = run_function_checks(functions_df)
function_results.to_csv(DOCS_DIR / 'timesfm_function_check_results.csv', index=False)
function_results.head(40)



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.4.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 758, in start
    self.io_loop.start()
  Fi

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.4.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 758, in start
    self.io_loop.start()
  Fi

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.4.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 758, in start
    self.io_loop.start()
  Fi

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.4.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 758, in start
    self.io_loop.start()
  Fi

AttributeError: _ARRAY_API not found

,id,path,module,function_name,status,error_type,error_message,is_callable,signature,no_required_args,invoke_status,invoke_error
0,16,timesfm.time_features._distance_to_holiday,timesfm.time_features,_distance_to_holiday,error,ImportError,numpy.core.multiarray failed to import,None,None,None,not_tried,None
1,37,timesfm.pytorch_patched_decoder.create_quantiles,timesfm.pytorch_patched_decoder,create_quantiles,ok,None,None,True,() -> list[float],True,ok,None
2,39,timesfm.pytorch_patched_decoder._masked_mean_std,timesfm.pytorch_patched_decoder,_masked_mean_std,ok,None,None,True,"(inputs: torch.Tensor, padding: torch.Tensor) ...",False,not_tried,None
3,40,timesfm.pytorch_patched_decoder._shift_padded_seq,timesfm.pytorch_patched_decoder,_shift_padded_seq,ok,None,None,True,"(mask: torch.Tensor, seq: torch.Tensor) -> tor...",False,not_tried,None
4,41,timesfm.pytorch_patched_decoder.get_large_nega...,timesfm.pytorch_patched_decoder,get_large_negative_number,ok,None,None,True,(dtype: torch.dtype) -> torch.Tensor,False,not_tried,None
5,42,timesfm.pytorch_patched_decoder.apply_mask_to_...,timesfm.pytorch_patched_decoder,apply_mask_to_logits,ok,None,None,True,"(logits: torch.Tensor, mask: torch.Tensor) -> ...",False,not_tried,None
6,43,timesfm.pytorch_patched_decoder.convert_paddin...,timesfm.pytorch_patched_decoder,convert_paddings_to_mask,ok,None,None,True,"(paddings: torch.Tensor, dtype: torch.dtype = ...",False,not_tried,None
7,44,timesfm.pytorch_patched_decoder.causal_mask,timesfm.pytorch_patched_decoder,causal_mask,ok,None,None,True,(input_t: torch.Tensor) -> torch.Tensor,False,not_tried,None
8,45,timesfm.pytorch_patched_decoder.merge_masks,timesfm.pytorch_patched_decoder,merge_masks,ok,None,None,True,"(a: torch.Tensor, b: torch.Tensor) -> torch.Te...",False,not_tried,None
9,103,timesfm.xreg_lib._unnest,timesfm.xreg_lib,_unnest,error,ImportError,numpy.core.multiarray failed to import,None,None,None,not_tried,None


### 5.2 Functions 単体深掘り


In [10]:
function_target = 'timesfm.timesfm_base.freq_map'
function_results[function_results['path'] == function_target]


,id,path,module,function_name,status,error_type,error_message,is_callable,signature,no_required_args,invoke_status,invoke_error
17,162,timesfm.timesfm_base.freq_map,timesfm.timesfm_base,freq_map,ok,None,None,True,(freq: str),False,not_tried,None


## 6. External 章


### 6.1 External 全件実行

外部依存（external）を import 可能か確認します。
`external path` が階層名の場合はトップレベル import も併せて確認します。


In [11]:
def run_external_checks(df: pd.DataFrame) -> pd.DataFrame:
    rows = []
    for r in df.itertuples(index=False):
        raw = str(r.Name)
        full = str(r.Path)
        primary = full if full else raw
        top = primary.split('.')[0]

        imp_primary = safe_import(primary)
        imp_top = safe_import(top) if top != primary else {'status': None, 'error_type': None, 'error_message': None}

        rows.append({
            'id': r.ID,
            'name': raw,
            'path': full,
            'module': r.Module,
            'import_target': primary,
            'primary_status': imp_primary['status'],
            'primary_error_type': imp_primary['error_type'],
            'primary_error_message': imp_primary['error_message'],
            'top_level_target': top,
            'top_level_status': imp_top['status'],
            'top_level_error_type': imp_top['error_type'],
            'top_level_error_message': imp_top['error_message'],
        })
    return pd.DataFrame(rows)

external_results = run_external_checks(externals_df)
external_results.to_csv(DOCS_DIR / 'timesfm_external_check_results.csv', index=False)
external_results.head(50)



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.4.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 758, in start
    self.io_loop.start()
  Fi

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.4.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 758, in start
    self.io_loop.start()
  Fi

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.4.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 758, in start
    self.io_loop.start()
  Fi

...


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.4.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 758, in start
    self.io_loop.start()
  Fi

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.4.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 758, in start
    self.io_loop.start()
  Fi

AttributeError: _ARRAY_API not found

,id,name,path,module,import_target,primary_status,primary_error_type,primary_error_message,top_level_target,top_level_status,top_level_error_type,top_level_error_message
0,10,absl,absl,timesfm.data_loader,absl,ok,None,None,absl,None,None,None
1,11,numpy,numpy,timesfm.data_loader,numpy,ok,None,None,numpy,None,None,None
2,12,pandas,pandas,timesfm.data_loader,pandas,ok,None,None,pandas,None,None,None
3,13,preprocessing,sklearn.preprocessing,timesfm.data_loader,sklearn.preprocessing,error,ImportError,numpy.core.multiarray failed to import,sklearn,error,ImportError,numpy.core.multiarray failed to import
4,14,tensorflow,tensorflow,timesfm.data_loader,tensorflow,ok,None,None,tensorflow,None,None,None
5,28,numpy,numpy,timesfm.time_features,numpy,ok,None,None,numpy,None,None,None
6,29,pandas,pandas,timesfm.time_features,pandas,ok,None,None,pandas,None,None,None
7,30,holiday,pandas.tseries.holiday,timesfm.time_features,pandas.tseries.holiday,ok,None,None,pandas,ok,None,None
8,31,offsets,pandas.tseries.offsets,timesfm.time_features,pandas.tseries.offsets,ok,None,None,pandas,ok,None,None
9,32,preprocessing,sklearn.preprocessing,timesfm.time_features,sklearn.preprocessing,error,ImportError,numpy.core.multiarray failed to import,sklearn,error,ImportError,numpy.core.multiarray failed to import


### 6.2 External 単体深掘り


In [12]:
external_target = 'jax'
external_results[(external_results['import_target'] == external_target) | (external_results['top_level_target'] == external_target)]


,id,name,path,module,import_target,primary_status,primary_error_type,primary_error_message,top_level_target,top_level_status,top_level_error_type,top_level_error_message
23,95,jax,jax,timesfm.timesfm_jax,jax,ok,None,None,jax,None,None,None
24,96,numpy,jax.numpy,timesfm.timesfm_jax,jax.numpy,ok,None,None,jax,ok,None,None
33,116,jax,jax,timesfm.xreg_lib,jax,ok,None,None,jax,None,None,None
34,117,numpy,jax.numpy,timesfm.xreg_lib,jax.numpy,ok,None,None,jax,ok,None,None
40,144,jax,jax,timesfm.patched_decoder,jax,ok,None,None,jax,None,None,None
41,145,numpy,jax.numpy,timesfm.patched_decoder,jax.numpy,ok,None,None,jax,ok,None,None


## 7. Methods/Props 章


### 7.1 Methods 全件実行

各 method について class からの解決、callable判定、signature取得を検証します。


In [13]:
def resolve_method(path: str):
    module_path, class_name, member_name = parse_class_and_member(path)
    if module_path is None:
        return {'status': 'error', 'error_type': 'ValueError', 'error_message': 'invalid method path'}, None, None
    imp = safe_import(module_path)
    if imp['module'] is None:
        return imp, None, None
    try:
        cls = getattr(imp['module'], class_name)
    except Exception as exc:  # noqa: BLE001
        return {
            'status': 'error',
            'module': imp['module'],
            'error_type': type(exc).__name__,
            'error_message': str(exc),
        }, None, None
    try:
        member = getattr(cls, member_name)
        return {'status': 'ok', 'module': imp['module'], 'error_type': None, 'error_message': None}, cls, member
    except Exception as exc:  # noqa: BLE001
        return {
            'status': 'error',
            'module': imp['module'],
            'error_type': type(exc).__name__,
            'error_message': str(exc),
        }, cls, None


def run_method_checks(df: pd.DataFrame) -> pd.DataFrame:
    rows = []
    for r in df.itertuples(index=False):
        imp, cls, member = resolve_method(str(r.Path))
        rec = {
            'id': r.ID,
            'path': r.Path,
            'module': r.Module,
            'method_name': r.Name,
            'status': imp['status'],
            'error_type': imp.get('error_type'),
            'error_message': imp.get('error_message'),
            'class_resolved': cls is not None,
            'member_resolved': member is not None,
            'member_type': None,
            'is_callable': None,
            'signature': None,
        }
        if member is not None:
            rec['member_type'] = type(member).__name__
            rec['is_callable'] = callable(member)
            try:
                rec['signature'] = str(inspect.signature(member))
            except Exception:
                rec['signature'] = None
        rows.append(rec)
    return pd.DataFrame(rows)

method_results = run_method_checks(methods_df)
method_results.to_csv(DOCS_DIR / 'timesfm_method_check_results.csv', index=False)
method_results.head(50)



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.4.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 758, in start
    self.io_loop.start()
  Fi

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.4.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 758, in start
    self.io_loop.start()
  Fi

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.4.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 758, in start
    self.io_loop.start()
  Fi

...


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.4.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 758, in start
    self.io_loop.start()
  Fi

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.4.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 758, in start
    self.io_loop.start()
  Fi

AttributeError: _ARRAY_API not found

,id,path,module,method_name,status,error_type,error_message,class_resolved,member_resolved,member_type,is_callable,signature
0,3,timesfm.data_loader.TimeSeriesdata.__init__,timesfm.data_loader,__init__,error,ImportError,numpy.core.multiarray failed to import,False,False,None,None,None
1,4,timesfm.data_loader.TimeSeriesdata._get_cat_cols,timesfm.data_loader,_get_cat_cols,error,ImportError,numpy.core.multiarray failed to import,False,False,None,None,None
2,5,timesfm.data_loader.TimeSeriesdata._normalize_...,timesfm.data_loader,_normalize_data,error,ImportError,numpy.core.multiarray failed to import,False,False,None,None,None
3,6,timesfm.data_loader.TimeSeriesdata.train_gen,timesfm.data_loader,train_gen,error,ImportError,numpy.core.multiarray failed to import,False,False,None,None,None
4,7,timesfm.data_loader.TimeSeriesdata.test_val_gen,timesfm.data_loader,test_val_gen,error,ImportError,numpy.core.multiarray failed to import,False,False,None,None,None
5,8,timesfm.data_loader.TimeSeriesdata._get_featur...,timesfm.data_loader,_get_features_and_ts,error,ImportError,numpy.core.multiarray failed to import,False,False,None,None,None
6,9,timesfm.data_loader.TimeSeriesdata.tf_dataset,timesfm.data_loader,tf_dataset,error,ImportError,numpy.core.multiarray failed to import,False,False,None,None,None
7,18,timesfm.time_features.TimeCovariates.__init__,timesfm.time_features,__init__,error,ImportError,numpy.core.multiarray failed to import,False,False,None,None,None
8,19,timesfm.time_features.TimeCovariates._minute_o...,timesfm.time_features,_minute_of_hour,error,ImportError,numpy.core.multiarray failed to import,False,False,None,None,None
9,20,timesfm.time_features.TimeCovariates._hour_of_day,timesfm.time_features,_hour_of_day,error,ImportError,numpy.core.multiarray failed to import,False,False,None,None,None


### 7.2 Props 生成・確認（class introspection）


In [14]:
def collect_properties(class_df: pd.DataFrame) -> pd.DataFrame:
    rows = []
    for r in class_df.itertuples(index=False):
        imp, cls = resolve_class(str(r.Path))
        if cls is None:
            rows.append({
                'class_path': r.Path,
                'status': imp['status'],
                'property_name': None,
                'property_doc_head': None,
            })
            continue
        props = []
        for name, value in inspect.getmembers(cls):
            if isinstance(value, property):
                props.append((name, ((value.__doc__ or '').strip().split("\n")[0])[:160]))
        if not props:
            rows.append({
                'class_path': r.Path,
                'status': 'ok',
                'property_name': None,
                'property_doc_head': None,
            })
        else:
            for name, doc in props:
                rows.append({
                    'class_path': r.Path,
                    'status': 'ok',
                    'property_name': name,
                    'property_doc_head': doc,
                })
    return pd.DataFrame(rows)

prop_results = collect_properties(classes_df)
prop_results.to_csv(DOCS_DIR / 'timesfm_property_check_results.csv', index=False)
prop_results.head(50)


SyntaxError: unterminated string literal (detected at line 16) (1330423120.py, line 16)

### 7.3 Methods/Props 単体深掘り


In [15]:
method_target = 'timesfm.timesfm_base.TimesFmBase.forecast_on_df'
property_class_target = 'timesfm.timesfm_base.TimesFmHparams'

print('method detail')
display(method_results[method_results['path'] == method_target])
print('property detail')
display(prop_results[prop_results['class_path'] == property_class_target])


method detail


,id,path,module,method_name,status,error_type,error_message,class_resolved,member_resolved,member_type,is_callable,signature
75,178,timesfm.timesfm_base.TimesFmBase.forecast_on_df,timesfm.timesfm_base,forecast_on_df,ok,None,None,True,True,function,True,"(self, inputs: pandas.core.frame.DataFrame, fr..."


property detail


NameError: name 'prop_results' is not defined

## 8. カテゴリ横断サマリ


In [16]:
_ok_like = {'ok', 'skipped_optional_dependency'}
summary = {
    'modules_ok_or_skipped': int(module_results['status'].isin(_ok_like).sum()),
    'modules_total': int(len(module_results)),
    'classes_ok_or_skipped': int(class_results['status'].isin(_ok_like).sum()),
    'classes_total': int(len(class_results)),
    'functions_ok_or_skipped': int(function_results['status'].isin(_ok_like).sum()),
    'functions_total': int(len(function_results)),
    'externals_ok_or_skipped': int(((external_results['primary_status'].isin(_ok_like)) | (external_results['top_level_status'].isin(_ok_like))).sum()),
    'externals_total': int(len(external_results)),
    'methods_ok_or_skipped': int(method_results['status'].isin(_ok_like).sum()),
    'methods_total': int(len(method_results)),
    'props_rows': int(len(prop_results)),
}
summary


NameError: name 'module_results' is not defined

In [17]:
out_summary = pd.DataFrame([
    {'category': 'modules', 'ok_or_skipped': summary['modules_ok_or_skipped'], 'total': summary['modules_total']},
    {'category': 'classes', 'ok_or_skipped': summary['classes_ok_or_skipped'], 'total': summary['classes_total']},
    {'category': 'functions', 'ok_or_skipped': summary['functions_ok_or_skipped'], 'total': summary['functions_total']},
    {'category': 'externals', 'ok_or_skipped': summary['externals_ok_or_skipped'], 'total': summary['externals_total']},
    {'category': 'methods', 'ok_or_skipped': summary['methods_ok_or_skipped'], 'total': summary['methods_total']},
])
out_summary['ok_or_skipped_rate'] = (out_summary['ok_or_skipped'] / out_summary['total']).round(4)
out_summary.to_csv(DOCS_DIR / 'timesfm_category_summary.csv', index=False)
out_summary


NameError: name 'summary' is not defined

## 9. 失敗項目の抽出（再実行対象）


In [18]:
_exclude = {'ok', 'skipped_optional_dependency'}

failed_modules = module_results[~module_results['status'].isin(_exclude)]
failed_classes = class_results[~class_results['status'].isin(_exclude)]
failed_functions = function_results[~function_results['status'].isin(_exclude)]
failed_externals = external_results[(~external_results['primary_status'].isin(_exclude)) & (~external_results['top_level_status'].isin(_exclude))]
failed_methods = method_results[~method_results['status'].isin(_exclude)]

optional_skipped = {
    'modules': module_results[module_results['status'] == 'skipped_optional_dependency'][['module', 'missing_optional_dep', 'error_message']].head(30).to_dict(orient='records'),
    'classes': class_results[class_results['status'] == 'skipped_optional_dependency'][['path', 'error_message']].head(30).to_dict(orient='records'),
    'functions': function_results[function_results['status'] == 'skipped_optional_dependency'][['path', 'error_message']].head(30).to_dict(orient='records'),
    'methods': method_results[method_results['status'] == 'skipped_optional_dependency'][['path', 'error_message']].head(30).to_dict(orient='records'),
}

print('hard failed counts:', len(failed_modules), len(failed_classes), len(failed_functions), len(failed_externals), len(failed_methods))

{
    'hard_failures': {
        'modules': failed_modules[['module', 'error_type', 'error_message']].head(20).to_dict(orient='records'),
        'classes': failed_classes[['path', 'error_type', 'error_message']].head(20).to_dict(orient='records'),
        'functions': failed_functions[['path', 'error_type', 'error_message']].head(20).to_dict(orient='records'),
        'externals': failed_externals[['import_target', 'primary_error_type', 'primary_error_message']].head(20).to_dict(orient='records'),
        'methods': failed_methods[['path', 'error_type', 'error_message']].head(20).to_dict(orient='records'),
    },
    'optional_skipped': optional_skipped,
}


NameError: name 'module_results' is not defined

## 10. 実行パイプライン（ロード/特徴量/モデル作成/予測/保存/解析）

この章は実際に処理を流すためのセルです。  
- DBロード: `dataset.loto_y_ts`（失敗時は合成データへフォールバック）  
- 特徴量生成: カレンダー特徴量  
- モデル作成: TimesFM（checkpoint 必須）  
- 予測: `forecast_on_df`  
- 保存: CSV/JSON/モデルメタ  
- 解析: 予測要約統計


In [ ]:
from pathlib import Path
import os
import json

from src.model.timesfm import (
    DbConfig,
    TimesFmRuntimeConfig,
    load_dataset_for_timesfm,
    generate_features,
    create_timesfm_model,
    run_forecast,
    analyze_predictions,
    save_runtime_artifacts,
)

RUNTIME_OUT = ROOT / 'outputs/timesfm_runtime'
RUNTIME_OUT.mkdir(parents=True, exist_ok=True)
print('RUNTIME_OUT =', RUNTIME_OUT)


In [ ]:
# ここを設定すると本実行できます（どちらか片方は必須）
# os.environ['TIMESFM_CKPT_PATH'] = '/path/to/checkpoint'
# os.environ['TIMESFM_HF_REPO_ID'] = 'google/timesfm-2.0-500m-pytorch'

runtime_cfg = TimesFmRuntimeConfig(
    freq='D',
    value_name='y',
    horizon_len=16,
    context_len=128,
    per_core_batch_size=32,
    backend='cpu',
    checkpoint_path=os.getenv('TIMESFM_CKPT_PATH'),
    huggingface_repo_id=os.getenv('TIMESFM_HF_REPO_ID'),
    checkpoint_version='torch',
    checkpoint_local_dir=str(RUNTIME_OUT / 'hf_cache'),
)
runtime_cfg


In [ ]:
data_result = load_dataset_for_timesfm(db_config=DbConfig.from_env(), limit=5000)
y_df = data_result['y_df']
features_df = generate_features(y_df)

print('data source =', data_result['source'])
print('data rows =', len(y_df), 'unique_id =', y_df['unique_id'].nunique())
if data_result['error']:
    print('data fallback reason =', data_result['error'])

display(y_df.head())
display(features_df.head())


In [ ]:
model_result = create_timesfm_model(runtime_cfg)
print('model status =', model_result['status'])
if model_result['error']:
    print('model error =', model_result['error'])

timesfm_model = model_result['model']


In [ ]:
if timesfm_model is not None:
    forecast_result = run_forecast(timesfm_model, y_df, freq=runtime_cfg.freq, value_name=runtime_cfg.value_name)
else:
    forecast_result = {
        'status': 'skipped',
        'pred_df': None,
        'error': 'model is None (checkpoint not set or load failed).',
    }

print('forecast status =', forecast_result['status'])
if forecast_result['error']:
    print('forecast error =', forecast_result['error'])

pred_df = forecast_result['pred_df']
if pred_df is not None:
    display(pred_df.head())


In [ ]:
if pred_df is not None:
    analysis_result = analyze_predictions(pred_df, y_df=y_df)
else:
    analysis_result = {
        'status': 'skipped',
        'reason': 'pred_df is None',
        'model_error': model_result.get('error'),
        'forecast_error': forecast_result.get('error'),
    }
analysis_result


In [ ]:
saved_files = save_runtime_artifacts(
    output_dir=RUNTIME_OUT,
    runtime=runtime_cfg,
    y_df=y_df,
    features_df=features_df,
    pred_df=pred_df,
    analysis=analysis_result,
    model_result=model_result,
)
saved_files


In [ ]:
# 可能なら torch state_dict も保存（best effort）
model_state_saved = {'status': 'skipped', 'path': None, 'error': None}
if timesfm_model is not None:
    try:
        import torch
        internal_model = getattr(timesfm_model, '_model', None)
        if internal_model is not None and hasattr(internal_model, 'state_dict'):
            model_state_path = RUNTIME_OUT / 'timesfm_torch_state_dict.pt'
            torch.save(internal_model.state_dict(), model_state_path)
            model_state_saved = {'status': 'ok', 'path': str(model_state_path), 'error': None}
        else:
            model_state_saved = {'status': 'skipped', 'path': None, 'error': 'internal _model/state_dict not available'}
    except Exception as exc:  # noqa: BLE001
        model_state_saved = {'status': 'error', 'path': None, 'error': f'{type(exc).__name__}: {exc}'}
model_state_saved


### 10.1 実行結果の回収

- `outputs/timesfm_runtime/timesfm_input.csv`  
- `outputs/timesfm_runtime/timesfm_features.csv`  
- `outputs/timesfm_runtime/timesfm_predictions.csv`（予測成功時）  
- `outputs/timesfm_runtime/timesfm_runtime_config.json`  
- `outputs/timesfm_runtime/timesfm_model_meta.json`  
- `outputs/timesfm_runtime/timesfm_analysis.json`  
- `outputs/timesfm_runtime/timesfm_torch_state_dict.pt`（保存可能時）
